In [4]:
import pandas as pd
import numpy as np

ITEM_COUNT = 10
ITEM_COLUMNS = []

for i in range(ITEM_COUNT):
    ITEM_COLUMNS.append('item' + str(i + 1))
    pass
user_ratings_df = pd.DataFrame(columns = ['user'] + ITEM_COLUMNS)
user1 = pd.Series(['u1', 99, 10, 80, 50, 40, 34, 99, 23, 10, 17], index = ['user'] + ITEM_COLUMNS)
user2 = pd.Series(['u2', 12, 80, 13, 87, 10, 93, 10, 80, 87, 90], index = ['user'] + ITEM_COLUMNS)
user3 = pd.Series(['u3', 50, 55, 65, 75, 45, 48, 56, 52, 59, 62], index = ['user'] + ITEM_COLUMNS)
user4 = pd.Series(['u4', 95, 12, 85, 49, 39, 32, 88, 12, 9, 21], index = ['user'] + ITEM_COLUMNS)
user5 = pd.Series(['u5', 49, 65, 54, 49, 34, 65, 67, 48, 52, 59], index = ['user'] + ITEM_COLUMNS)
user6 = pd.Series(['u6', 15, 90, 15, 95, 17, 94, 12, 85, 90, 93], index = ['user'] + ITEM_COLUMNS)
user7 = pd.Series(['u7', 11, 89, 17, 93, 19, 92, 16, 94, 98, 92], index = ['user'] + ITEM_COLUMNS)
user8 = pd.Series(['u8', 94, 11, 89, 48, 42, 30, 92, 16, 12, 25], index = ['user'] + ITEM_COLUMNS)
user9 = pd.Series(['u9', 19, 88, 21, 98, 9, 90, 11, 97, 89, 86], index = ['user'] + ITEM_COLUMNS)
user10 = pd.Series(['u10', 66, 43, 45, 34, 21, 45, 75, 32, 12, 55], index = ['user'] + ITEM_COLUMNS)

user_ratings_df = pd.concat([user_ratings_df, user1.to_frame().T, user2.to_frame().T, user3.to_frame().T, \
            user4.to_frame().T, user5.to_frame().T, user6.to_frame().T, user7.to_frame().T, \
            user8.to_frame().T, user9.to_frame().T, user10.to_frame().T], \
            ignore_index = True, axis = 0)

print(user_ratings_df)

  user item1 item2 item3 item4 item5 item6 item7 item8 item9 item10
0   u1    99    10    80    50    40    34    99    23    10     17
1   u2    12    80    13    87    10    93    10    80    87     90
2   u3    50    55    65    75    45    48    56    52    59     62
3   u4    95    12    85    49    39    32    88    12     9     21
4   u5    49    65    54    49    34    65    67    48    52     59
5   u6    15    90    15    95    17    94    12    85    90     93
6   u7    11    89    17    93    19    92    16    94    98     92
7   u8    94    11    89    48    42    30    92    16    12     25
8   u9    19    88    21    98     9    90    11    97    89     86
9  u10    66    43    45    34    21    45    75    32    12     55


In [23]:
### Simple approach - calculate averages from all users and use it to propose it to John
john_entry = pd.Series(['John'] + np.empty(ITEM_COUNT).tolist(), index = ['user'] + ITEM_COLUMNS)
print(john_entry)

for i in range(ITEM_COUNT):
    john_entry[i+1] = np.average(user_ratings_df.iloc[:, i + 1]) 

print(john_entry)

user      John
item1      0.0
item2      0.0
item3      0.0
item4      0.0
item5      0.0
item6      0.0
item7      0.0
item8      0.0
item9      0.0
item10     0.0
dtype: object
user      John
item1     51.0
item2     54.3
item3     48.4
item4     67.8
item5     27.6
item6     62.3
item7     52.6
item8     53.9
item9     51.8
item10    60.0
dtype: object


In [61]:
### Assume we've got information about user location, from registration log and IP localization for example(in form of latitude and longitude)
generated_coords = np.random.rand(10, 2) * [180, 360]  - [90, 180]
john_coords = np.random.rand(1, 2) * [180, 360]  - [90, 180]
user_ratings_df['latitude'] = generated_coords[:, 0]
user_ratings_df['longitude'] = generated_coords[:, 1]
john_entry['latitude'] = john_coords[:, 0][0]
john_entry['longitude'] = john_coords[:, 1][0]
print(user_ratings_df)
print(john_entry)

  user item1 item2 item3 item4 item5 item6 item7 item8 item9 item10  \
0   u1    99    10    80    50    40    34    99    23    10     17   
1   u2    12    80    13    87    10    93    10    80    87     90   
2   u3    50    55    65    75    45    48    56    52    59     62   
3   u4    95    12    85    49    39    32    88    12     9     21   
4   u5    49    65    54    49    34    65    67    48    52     59   
5   u6    15    90    15    95    17    94    12    85    90     93   
6   u7    11    89    17    93    19    92    16    94    98     92   
7   u8    94    11    89    48    42    30    92    16    12     25   
8   u9    19    88    21    98     9    90    11    97    89     86   
9  u10    66    43    45    34    21    45    75    32    12     55   

    latitude   longitude  similarity_to_John  
0 -18.257841  -77.766464           -0.983686  
1  81.417865  -49.532235            0.983686  
2  57.137546 -120.042124           -0.983686  
3  13.287418  -24.407492      

In [62]:
### Now let's calculate the cosine similarity of the location and based upon it, calculate potential ratings 
from scipy import spatial
user_ratings_df['similarity_to_John'] = user_ratings_df.apply(lambda entry: 1 - spatial.distance.cosine([entry['longitude'], entry['latitude']], [john_entry['longitude'], john_entry['latitude']]), axis=1)
print(user_ratings_df)

  user item1 item2 item3 item4 item5 item6 item7 item8 item9 item10  \
0   u1    99    10    80    50    40    34    99    23    10     17   
1   u2    12    80    13    87    10    93    10    80    87     90   
2   u3    50    55    65    75    45    48    56    52    59     62   
3   u4    95    12    85    49    39    32    88    12     9     21   
4   u5    49    65    54    49    34    65    67    48    52     59   
5   u6    15    90    15    95    17    94    12    85    90     93   
6   u7    11    89    17    93    19    92    16    94    98     92   
7   u8    94    11    89    48    42    30    92    16    12     25   
8   u9    19    88    21    98     9    90    11    97    89     86   
9  u10    66    43    45    34    21    45    75    32    12     55   

    latitude   longitude  similarity_to_John  
0 -18.257841  -77.766464            0.893583  
1  81.417865  -49.532235           -0.149024  
2  57.137546 -120.042124            0.417250  
3  13.287418  -24.407492      